In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import gmean, ttest_ind
from statsmodels.stats.multitest import multipletests
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.stats import f_oneway
import scipy.stats as ss
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm


# transcriptome
from statsmodels.formula.api import ols

In [17]:
Transcriptome = pd.read_excel("/Users/mortezaabyadeh/Desktop/whole_RNAseq.xlsx")

In [3]:
Transcriptome.head()

,Gene,CTRL-1,CTRL-2,CTRL-3,Eto-1,Eto-2,Eto-3,FTY-1,FTY-2,FTY-3,FTY-Eto-1,FTY-Eto-2,FTY-Eto-3
0,A1BG,9.226356,9.579789,11.768253,12.130163,8.907069,7.734345,7.045783,10.049876,10.108732,11.637480,22.065324,9.726837
1,A1BG-AS1,70.735396,79.512247,80.238087,59.548075,59.009335,73.476277,50.201203,56.279304,87.609007,69.824882,76.225666,73.923962
2,A2M,243.985857,235.662806,219.317438,344.055545,365.189848,377.693846,171.740957,228.132179,269.566175,235.923463,262.777954,182.864539
3,A4GALT,331.123664,319.964948,353.047584,395.884425,312.860815,309.373798,432.434922,423.099767,551.487466,322.675589,338.000650,372.537863
4,AAAS,745.284531,777.878855,766.006272,632.973984,616.814561,665.153667,802.338522,848.209510,663.806706,752.204406,756.238843,804.409433


In [6]:
expression_data = Transcriptome.iloc[:, 1:]
genes = Transcriptome.iloc[:, 0]
expression_data.head()

,CTRL-1,CTRL-2,CTRL-3,Eto-1,Eto-2,Eto-3,FTY-1,FTY-2,FTY-3,FTY-Eto-1,FTY-Eto-2,FTY-Eto-3
0,9.226356,9.579789,11.768253,12.130163,8.907069,7.734345,7.045783,10.049876,10.108732,11.637480,22.065324,9.726837
1,70.735396,79.512247,80.238087,59.548075,59.009335,73.476277,50.201203,56.279304,87.609007,69.824882,76.225666,73.923962
2,243.985857,235.662806,219.317438,344.055545,365.189848,377.693846,171.740957,228.132179,269.566175,235.923463,262.777954,182.864539
3,331.123664,319.964948,353.047584,395.884425,312.860815,309.373798,432.434922,423.099767,551.487466,322.675589,338.000650,372.537863
4,745.284531,777.878855,766.006272,632.973984,616.814561,665.153667,802.338522,848.209510,663.806706,752.204406,756.238843,804.409433


In [7]:

groups = [expression_data.iloc[:, i:i+3] for i in range(0, len(expression_data.columns), 3)]


P_Val = []
for i in range(len(expression_data)):
    data_i = pd.DataFrame(expression_data.iloc[i, :])  # Convert to DataFrame
    group_i = [data_i.iloc[j:j+3, :] for j in range(0, len(data_i), 3)]  # Access rows for each group
    p_value = f_oneway(*group_i)
    P_Val.append(p_value)


p_values_df = pd.DataFrame(P_Val, index=genes, columns=['F_value', 'p_value'])  # Adjusted to include F_value


#print(p_values_df)

In [11]:
print(p_values_df.head(),
Transcriptome.head())

                       F_value                  p_value
Gene                                                   
A1BG      [1.3526604488608214]    [0.32461635875749295]
A1BG-AS1  [0.9541684720153194]     [0.4594933165639652]
A2M       [12.079885898012428]  [0.0024334229401704248]
A4GALT     [5.971664117462256]    [0.01938335319657668]
AAAS      [4.6637480167410805]   [0.036258477760605014]        Gene      CTRL-1      CTRL-2      CTRL-3       Eto-1       Eto-2  \
0      A1BG    9.226356    9.579789   11.768253   12.130163    8.907069   
1  A1BG-AS1   70.735396   79.512247   80.238087   59.548075   59.009335   
2       A2M  243.985857  235.662806  219.317438  344.055545  365.189848   
3    A4GALT  331.123664  319.964948  353.047584  395.884425  312.860815   
4      AAAS  745.284531  777.878855  766.006272  632.973984  616.814561   

        Eto-3       FTY-1       FTY-2       FTY-3   FTY-Eto-1   FTY-Eto-2  \
0    7.734345    7.045783   10.049876   10.108732   11.637480   22.065324   
1  

In [14]:
p_values_df.to_excel("/Users/mortezaabyadeh/Desktop/p_values.xlsx")

# I just made them like dataset in the Excel and then imported again

In [16]:
p = pd.read_excel("/Users/mortezaabyadeh/Desktop/p_values.xlsx")
p.head()

,Gene,p_value
0,A1BG,0.324616
1,A1BG-AS1,0.459493
2,A2M,0.002433
3,A4GALT,0.019383
4,AAAS,0.036258


In [18]:
Transcriptome["ANOVA"] = p["p_value"]

In [19]:
Transcriptome.head()

,Gene,CTRL-1,CTRL-2,CTRL-3,Eto-1,Eto-2,Eto-3,FTY-1,FTY-2,FTY-3,FTY-Eto-1,FTY-Eto-2,FTY-Eto-3,ANOVA
0,A1BG,9.226356,9.579789,11.768253,12.130163,8.907069,7.734345,7.045783,10.049876,10.108732,11.637480,22.065324,9.726837,0.324616
1,A1BG-AS1,70.735396,79.512247,80.238087,59.548075,59.009335,73.476277,50.201203,56.279304,87.609007,69.824882,76.225666,73.923962,0.459493
2,A2M,243.985857,235.662806,219.317438,344.055545,365.189848,377.693846,171.740957,228.132179,269.566175,235.923463,262.777954,182.864539,0.002433
3,A4GALT,331.123664,319.964948,353.047584,395.884425,312.860815,309.373798,432.434922,423.099767,551.487466,322.675589,338.000650,372.537863,0.019383
4,AAAS,745.284531,777.878855,766.006272,632.973984,616.814561,665.153667,802.338522,848.209510,663.806706,752.204406,756.238843,804.409433,0.036258


In [20]:
Transcriptome.to_excel("/Users/mortezaabyadeh/Desktop/Transcript.xlsx")

In [ ]:
# use it in R to Make PCA